In [0]:
import numpy as np  #numpy
import pandas as pd  #pandas
import re
import warnings
warnings.filterwarnings('ignore')
import time
import datetime as dt

ad_operation_path = 'https://lwh-tenxun-test.oss-cn-shenzhen.aliyuncs.com/testA/ad_operation.csv?Expires=1557984966&OSSAccessKeyId=TMP.AgG_TFzSypSX20nT3DHD243rxSSuYKGdas9R9OGgCyqsJcEubKZJnNzY-8CqMC4CFQCYuRATpZYgFqpmmBCN1AcTDMWj3QIVAJg11hNqoRRtN1xVLj9fe2rk3vne&Signature=lnpHVAxuWc%2BMVUc1oTE%2BFyU0lGE%3D'
ad_static_feature_path = 'https://lwh-tenxun-test.oss-cn-shenzhen.aliyuncs.com/testA/ad_static_feature.csv?Expires=1558046221&OSSAccessKeyId=TMP.AgG_TFzSypSX20nT3DHD243rxSSuYKGdas9R9OGgCyqsJcEubKZJnNzY-8CqMC4CFQCYuRATpZYgFqpmmBCN1AcTDMWj3QIVAJg11hNqoRRtN1xVLj9fe2rk3vne&Signature=Qs3R0AgKa6J7yu3OM0LpfqWM9Xg%3D'
test_sample_path = 'https://lwh-tenxun-test.oss-cn-shenzhen.aliyuncs.com/testA/test_sample.csv?Expires=1558046246&OSSAccessKeyId=TMP.AgG_TFzSypSX20nT3DHD243rxSSuYKGdas9R9OGgCyqsJcEubKZJnNzY-8CqMC4CFQCYuRATpZYgFqpmmBCN1AcTDMWj3QIVAJg11hNqoRRtN1xVLj9fe2rk3vne&Signature=T70m1XD6oaZOa3PGSWzjTnqJ4%2Fw%3D'

ad_operation = pd.read_csv(ad_operation_path,sep='\t',names=['ad_id','Change_time','Operation_type',
                                                             'Modify_field','value_after_operation'])
ad_static_feature = pd.read_csv(ad_static_feature_path,sep='\t',names=['ad_id','Creation_time','ad_account_id','Product_id',
                                                                       'Product_Types','ad_industry_id','ad_size'])
test_sample = pd.read_csv(test_sample_path,sep='\t',names=['Sample_id','ad_id','Creation_time','ad_size','ad_industry_id','Product_Types',
                                                           'Product_id','ad_account_id','Delivery_period','Crowd_orientation','bid'])



In [0]:
test_sample['Creation_time'] = pd.to_datetime(test_sample['Creation_time'],unit='s')
ad_operation['Change_time'] = pd.to_datetime(ad_operation.Change_time, format='%Y%m%d%H%M%S', errors='coerce')
ad_static_feature['Creation_time'] = pd.to_datetime(ad_static_feature['Creation_time'],unit='s')

### 静态数据

In [0]:
ad_static_feature = ad_static_feature[~ad_static_feature['Product_id'].isin(['-1',-1.0])]

In [10]:

ad_static_feature = (ad_static_feature.set_index(['ad_id','Creation_time','ad_account_id','Product_id'
                                                  ,'Product_Types','ad_industry_id']).astype(str)
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)

ad_static_feature.head()

,ad_id,Creation_time,ad_account_id,Product_id,Product_Types,ad_industry_id,ad_size
0,1,2019-02-20 02:51:34,3838,8789,18,117,64
1,2,2019-02-18 03:21:00,12827,10418,18,117,36
2,3,2019-03-06 10:01:03,1520,3911,18,232,64
3,6,2019-02-16 11:46:49,22180,28082,18,117,36
4,8,2019-03-21 11:58:22,29275,342,18,232,40


In [0]:
ad_static_feature['ad_size'] = pd.to_numeric(ad_static_feature['ad_size'], errors='coerce')
ad_static_feature.ad_size = ad_static_feature.ad_size.fillna(ad_static_feature.ad_size.median())
ad_static_feature.ad_size = ad_static_feature.ad_size.astype(int)

In [0]:
ad_static_feature = (ad_static_feature.set_index(['ad_id','Creation_time','ad_account_id'
                                                  ,'Product_Types','ad_size']).astype(str)
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)

In [13]:
ad_static_feature = ad_static_feature.drop_duplicates(subset=['ad_id','Creation_time','ad_account_id','Product_Types','ad_size','ad_industry_id'] )
ad_static_feature = ad_static_feature.drop_duplicates(subset=['ad_id','Creation_time','ad_account_id','Product_Types','ad_size','Product_id'] )
ad_static_feature = ad_static_feature.dropna()
ad_static_feature.ad_industry_id = ad_static_feature.ad_industry_id.astype(int)
ad_static_feature.Product_id = ad_static_feature.Product_id.astype(int)

ad_static_feature.info()
ad_static_feature.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492384 entries, 0 to 492383
Data columns (total 7 columns):
ad_id             492384 non-null int64
Creation_time     492384 non-null datetime64[ns]
ad_account_id     492384 non-null int64
Product_Types     492384 non-null int64
ad_size           492384 non-null int64
Product_id        492384 non-null int64
ad_industry_id    492384 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 30.1 MB


,ad_id,Creation_time,ad_account_id,Product_Types,ad_size,Product_id,ad_industry_id
0,1,2019-02-20 02:51:34,3838,18,64,8789,117
1,2,2019-02-18 03:21:00,12827,18,36,10418,117
2,3,2019-03-06 10:01:03,1520,18,64,3911,232
3,6,2019-02-16 11:46:49,22180,18,36,28082,117
4,8,2019-03-21 11:58:22,29275,18,40,342,232


In [14]:
ad_static_feature['Creation_time'] = ad_static_feature['Creation_time'].dt.date
ad_static_feature.head()

,ad_id,Creation_time,ad_account_id,Product_Types,ad_size,Product_id,ad_industry_id
0,1,2019-02-20,3838,18,64,8789,117
1,2,2019-02-18,12827,18,36,10418,117
2,3,2019-03-06,1520,18,64,3911,232
3,6,2019-02-16,22180,18,36,28082,117
4,8,2019-03-21,29275,18,40,342,232


### 操作数据

In [15]:
ad_operation = ad_operation.replace({'Change_time':0},np.nan)
ad_operation = ad_operation[~ad_operation['value_after_operation'].isin(['0'])]
ad_operation = ad_operation.rename(columns={'Change_time':'Creation_time'})
ad_operation['Creation_time'] = ad_operation['Creation_time'].dt.date
ad_operation.head()

,ad_id,Creation_time,Operation_type,Modify_field,value_after_operation
0,593323,NaT,2,2,90
1,593323,NaT,2,3,all
2,593323,NaT,2,4,"281474976710655,281474976710655,28147497671065..."
4,593323,2019-02-18,1,2,90
5,593323,2019-02-18,1,1,1


### 合并

In [16]:
ad_data = ad_operation.combine_first(ad_static_feature)
ad_data = ad_data.dropna()
ad_data['Modify_field'] = ad_data['Modify_field'].astype(int)
ad_data['Operation_type'] = ad_data['Operation_type'].astype(int)
ad_data['Product_Types'] = ad_data['Product_Types'].astype(int)
ad_data['Product_id'] = ad_data['Product_id'].astype(int)
ad_data['ad_account_id'] = ad_data['ad_account_id'].astype(int)
ad_data['ad_id'] = ad_data['ad_id'].astype(int)
ad_data['ad_industry_id'] = ad_data['ad_industry_id'].astype(int)
ad_data['ad_size'] = ad_data['ad_size'].astype(int)
ad_data.head()

,Creation_time,Modify_field,Operation_type,Product_Types,Product_id,ad_account_id,ad_id,ad_industry_id,ad_size,value_after_operation
0,2019-02-20,2,2,18,8789,3838,593323,117,64,90
1,2019-02-18,3,2,18,10418,12827,593323,117,36,all
2,2019-03-06,4,2,18,3911,1520,593323,232,64,"281474976710655,281474976710655,28147497671065..."
4,2019-02-18,2,1,18,342,29275,593323,232,40,90
5,2019-02-18,1,1,18,16078,3934,593323,117,54,1
